In [ ]:
# -- Switch to postgres user

# sudo -u postgres psql

# -- Create database

# CREATE DATABASE restaurant_reviews;

# -- Create database user and grant all previlages

# CREATE USER admin WITH PASSWORD 'admin123';

# GRANT ALL PRIVILEGES ON DATABASE restaurant_reviews TO admin;



# -- Login using the new user:
# psql -U admin -d restaurant_reviews -h localhost


# -- Create the tables


# -- Restaurants table

# CREATE TABLE restaurants (
#     restaurant_id SERIAL PRIMARY KEY,
#     restaurant_name VARCHAR(255) UNIQUE
# );


# -- Reviews table

# CREATE TABLE reviews (
#     review_id SERIAL PRIMARY KEY,
#     restaurant_id INT REFERENCES restaurants(restaurant_id),
#     review_text TEXT,
#     rating INT,
#     review_date DATE,
#     source VARCHAR(50)
# );



# --check the tables

# \d


# -- List all tables

# \dt

# -- Describe a table (check schema)

# \d table_name


# -- Delete all records

# TRUNCATE TABLE table_name RESTART IDENTITY CASCADE;


In [2]:
# pip install psycopg2-binary
import pandas as pd
import psycopg2

In [3]:
# Read CSV files into pandas DataFrames
df_restaurants = pd.read_csv("../data/restaurants.csv")
df_reviews = pd.read_csv("../data/reviews.csv")

In [4]:
# -------------------------------------------------------
# Connect to PostgreSQL database
# -------------------------------------------------------

# Establish a connection to PostgreSQL using psycopg2
# You must make sure the database 'restaurant_reviews' is already created
conn = psycopg2.connect(
    host="localhost",              # PostgreSQL is running on your machine
    database="restaurant_reviews", # The database name you created
    user="admin",                  # Your PostgreSQL username
    password="admin123"            # Your PostgreSQL password
)

# Create a cursor object — used to execute SQL commands
cur = conn.cursor()

In [5]:
# -------------------------------------------------------
# Insert data into restaurants table
# -------------------------------------------------------

# Loop through each row in df_restaurants
for _, row in df_restaurants.iterrows():

    # Execute SQL INSERT for each row
    # ON CONFLICT DO NOTHING avoids errors if the primary key already exists
    cur.execute(
        """
        INSERT INTO restaurants (restaurant_id, restaurant_name)
        VALUES (%s, %s)
        ON CONFLICT DO NOTHING;
        """,
        (
            int(row["restaurant_id"]),   # Convert to integer
            row["restaurant_name"]       # Restaurant name as string
        )
    )


In [6]:

# -------------------------------------------------------
# Insert data into reviews table
# -------------------------------------------------------

# Loop through each row in df_reviews
for _, row in df_reviews.iterrows():

    # Insert each review into PostgreSQL
    cur.execute(
        """
        INSERT INTO reviews 
            (review_id, restaurant_id, review_text, rating, review_date, source)
        VALUES 
            (%s, %s, %s, %s, %s, %s)
        ON CONFLICT DO NOTHING;
        """,
        (
            int(row["review_id"]),       # Review ID (primary key)
            int(row["restaurant_id"]),   # Foreign key referencing restaurants table
            row["review_text"],          # Full text review
            int(row["rating"]),          # Numerical rating
            row["review_date"],          # Date string
            row["source"]                # Review source (e.g., "csv", "app", etc.)
        )
    )

# -------------------------------------------------------
# Save changes and close connection
# -------------------------------------------------------

conn.commit()   # Saves all INSERT operations permanently
cur.close()     # Close the cursor
conn.close()    # Close the database connection

print("Data inserted successfully!")


Data inserted successfully!
